In [0]:
# =====================================================
# USE CASE 2: Subscription Plan Analysis & Optimization
# Project: Viewer Churn Prediction for OTT Platforms
# Platform: Databricks (Free Edition)
# Author: Erugurala Teja (24MBMB19)
# =====================================================

# Step 1️⃣: Import Required Libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, length, regexp_replace, lower, avg, count
from pyspark.sql import functions as F

In [0]:
# Step 2️⃣: Initialize Spark Session
spark = SparkSession.builder.appName("OTT_Subscription_Analysis").getOrCreate()


In [0]:
# Step 3️⃣: Load the Dataset
data_path = "/Volumes/workspace/default/dataset/ott_reviews.csv"
df = spark.read.csv(data_path, header=True, inferSchema=True)
print("✅ Data successfully loaded!")
print(f"Total Records: {df.count()}")
df.printSchema()

✅ Data successfully loaded!
Total Records: 6000
root
 |-- app_name: string (nullable = true)
 |-- reviewId: string (nullable = true)
 |-- userName: string (nullable = true)
 |-- content: string (nullable = true)
 |-- score: string (nullable = true)
 |-- at: string (nullable = true)



In [0]:
# Step 4️⃣: Basic Data Cleaning (same as Use Case 1)
df_clean = df.dropna(subset=["content", "score"])
df_clean = df_clean.withColumn("content", regexp_replace("content", "[^a-zA-Z ]", " "))
df_clean = df_clean.withColumn("content", lower(col("content")))
df_clean = df_clean.filter(length(col("content")) > 20)

print("✅ Data cleaned successfully — showing sample:")
df_clean.select("app_name", "score", "content").show(5, truncate=False)


✅ Data cleaned successfully — showing sample:
+--------+--------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|app_name|score               |content                                                                                                                                                                                                                                                                                                                                                                                               

In [0]:
# Step 5️⃣: Safe Conversion of 'score' Column
from pyspark.sql.functions import expr
df_clean = df_clean.withColumn("score_num", expr("try_cast(score as double)"))
df_clean = df_clean.dropna(subset=["score_num"])

print("✅ Score column successfully converted to numeric!")


✅ Score column successfully converted to numeric!


In [0]:
# Step 6️⃣: Create Churn Label (consistent with Use Case 1)
df_final = df_clean.withColumn(
    "churn_label",
    when(col("score_num") <= 2, 1).when(col("score_num") >= 4, 0).otherwise(None)
).dropna(subset=["churn_label"])

print("✅ Target variable 'churn_label' created successfully!")
df_final.select("app_name", "score_num", "churn_label").show(5)


✅ Target variable 'churn_label' created successfully!
+--------+---------+-----------+
|app_name|score_num|churn_label|
+--------+---------+-----------+
| Netflix|      5.0|          0|
| Netflix|      1.0|          1|
| Netflix|      1.0|          1|
| Netflix|      1.0|          1|
| Netflix|      5.0|          0|
+--------+---------+-----------+
only showing top 5 rows


In [0]:
# =====================================================
# SPARK SQL SECTION (Core of Use Case 2)
# =====================================================

# Step 7️⃣: Create Temporary SQL View
df_final.createOrReplaceTempView("ott_reviews")

In [0]:
# Step 8️⃣: Analyze Average Rating per OTT Platform
avg_rating = spark.sql("""
SELECT app_name,
       ROUND(AVG(score_num), 2) AS avg_rating,
       COUNT(*) AS total_reviews
FROM ott_reviews
GROUP BY app_name
ORDER BY avg_rating DESC
""")
print("✅ Average Rating per Platform calculated!")
display(avg_rating)

✅ Average Rating per Platform calculated!


app_name,avg_rating,total_reviews
JioCinema,4.11,111
ZEE5,2.4,315
Netflix,2.36,603
Disney+ Hotstar,2.34,386
Amazon Prime Video,1.46,795
SonyLIV,1.37,679


In [0]:
# Step 9️⃣: Analyze Churn Rate per Platform
churn_rate = spark.sql("""
SELECT app_name,
       COUNT(*) AS total_users,
       SUM(CASE WHEN churn_label = 1 THEN 1 ELSE 0 END) AS churned_users,
       ROUND((SUM(CASE WHEN churn_label = 1 THEN 1 ELSE 0 END) / COUNT(*)) * 100, 2) AS churn_rate
FROM ott_reviews
GROUP BY app_name
ORDER BY churn_rate DESC
""")
print("✅ Churn Rate per Platform calculated!")
display(churn_rate)

✅ Churn Rate per Platform calculated!


app_name,total_users,churned_users,churn_rate
SonyLIV,679,621,91.46
Amazon Prime Video,795,722,90.82
Disney+ Hotstar,386,258,66.84
Netflix,603,399,66.17
ZEE5,315,204,64.76
JioCinema,111,22,19.82


In [0]:
# Step 🔟: Join Both Results into One Final Summary
final_summary = avg_rating.join(
    churn_rate,
    on="app_name",
    how="inner"
).select("app_name", "avg_rating", "total_reviews", "churn_rate")

print("✅ Final Platform Summary prepared!")
display(final_summary)

✅ Final Platform Summary prepared!


app_name,avg_rating,total_reviews,churn_rate
Amazon Prime Video,1.46,795,90.82
Disney+ Hotstar,2.34,386,66.84
JioCinema,4.11,111,19.82
ZEE5,2.4,315,64.76
SonyLIV,1.37,679,91.46
Netflix,2.36,603,66.17


In [0]:
# Step 1️⃣1️⃣: Save the Results to CSV
output_path = "/Volumes/workspace/default/dataset/ott_subscription_analysis"
final_summary.coalesce(1).write.mode("overwrite").option("header", True).csv(output_path)
print(f"✅ Results saved successfully to: {output_path}")


✅ Results saved successfully to: /Volumes/workspace/default/dataset/ott_subscription_analysis


In [0]:
# Step 1️⃣2️⃣: Humanized Insights
summary_pd = final_summary.toPandas()

top_app = summary_pd.sort_values("avg_rating", ascending=False).iloc[0]["app_name"]
lowest_churn_app = summary_pd.sort_values("churn_rate").iloc[0]["app_name"]
highest_churn_app = summary_pd.sort_values("churn_rate", ascending=False).iloc[0]["app_name"]

print("===============================================")
print("📊 HUMANIZED OUTPUT SUMMARY (USE CASE 2)")
print(f"Highest Rated OTT Platform: {top_app}")
print(f"Lowest Churn Rate: {lowest_churn_app}")
print(f"Highest Churn Rate: {highest_churn_app}")
print("===============================================")

📊 HUMANIZED OUTPUT SUMMARY (USE CASE 2)
Highest Rated OTT Platform: JioCinema
Lowest Churn Rate: JioCinema
Highest Churn Rate: SonyLIV


In [0]:
# =====================================================
# Step 1️⃣3️⃣: Export Downloadable CSV for Viva (Databricks Volumes Safe Way)
# =====================================================

# Reuse your safe, writable volume path
export_path = "/Volumes/workspace/default/dataset/ott_subscription_analysis_export"

# Write output to the volume
final_summary.coalesce(1).write.mode("overwrite").option("header", True).csv(export_path)

print("✅ Export completed successfully!")
print("📂 Download Instructions:")
print("1️⃣  Go to: Data ➜ Volumes ➜ workspace ➜ default ➜ dataset ➜ ott_subscription_analysis_export")
print("2️⃣  Inside, open the folder and find a file named:")
print("    part-00000-xxxx.csv  → right-click ➜ Download")
print("✅ This CSV contains your final subscription analysis results.")


✅ Export completed successfully!
📂 Download Instructions:
1️⃣  Go to: Data ➜ Volumes ➜ workspace ➜ default ➜ dataset ➜ ott_subscription_analysis_export
2️⃣  Inside, open the folder and find a file named:
    part-00000-xxxx.csv  → right-click ➜ Download
✅ This CSV contains your final subscription analysis results.
